# PySpark RDD

### 1. 准备工作

配置和启动 PySpark：

In [1]:
import findspark
findspark.init("/Users/xinby/Library/Spark")

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[*]").\
    appName("PySpark RDD").\
    getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
print(spark)
print(sc)

23/03/30 16:03:34 WARN Utils: Your hostname, XinBys-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.65.6.2 instead (on interface en0)
23/03/30 16:03:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/30 16:03:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/30 16:03:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
<SparkContext master=local[*] appName=PySpark RDD>


### 2. RDD

创建一个包含了数据的列表：

In [2]:
import math
vec = [math.sin(i + math.exp(i)) for i in range(100)]

将其转换为分布式数据结构： 

`sc = spark.sparkContext`

`sc.parallelize()`

In [3]:
dat = sc.parallelize(vec)
dat #dat 为 rdd 对象

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

`dat` 的类型是 RDD（Resilient Distributed Dataset），是一种类似于迭代器的结构，可以看作是某种数据类型的容器。例如，`dat` 代表了一些数字的集合。

类似于 Python 中原生的函数式编程工具，可以在 RDD 中使用 Map/Filter/Reduce 等操作。例如计算求和：

In [4]:
dat.reduce(lambda x, y: x + y) #加和容器内的elem

-2.2461678254937105

灵活使用 Map 函数计算均值：

- rdd 可以进行`map` `reduce` `filter` 的串联，顺序即是从左到右
- rdd 也可以无限套娃 -> 根本的原则在于只要返回的内容是一个新的rdd，就可以继续嵌套

In [5]:
dat.map(lambda x: (1, x)).reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))
n,sum,sqsum = dat.map(lambda x: (1, x, x*x)).reduce(lambda x, y: (x[0]+y[0],x[1]+y[1],x[2]+y[2]))
print(n,sum,sqsum)
var = (sqsum - n*sum)/(n-1)
print(var)


100 -2.2461678254937105 45.97765030189205
2.733277099507708


Filter 操作：

In [8]:
dat.filter(lambda x: x > 0).reduce(lambda x, y: x + y)

28.01568102822997

In [9]:
dat.filter(lambda x: x <= 0).reduce(lambda x, y: x + y)

-30.261848853723677

使用 `collect()` 函数可以将 RDD 中的数据*全部取出*返回给 Python，***但对于大型数据请谨慎操作！***

In [10]:
dat.collect()

[0.8414709848078965,
 -0.5452515566923345,
 0.035714265168052234,
 -0.8886479175586053,
 0.8876009615390265,
 0.5011099612213634,
 0.8530218378956966,
 -0.804086324216863,
 -0.9644551022640215,
 0.4721216528877472,
 0.9723105121477627,
 0.10237280614077034,
 0.7682074937713861,
 0.819078842327986,
 -0.7885252480190347,
 -0.8483650910372162,
 -0.24455653034314892,
 -0.8558519039634782,
 -0.10196456500793881,
 0.14618338451195675,
 0.9999672698820121,
 -0.07924816445805015,
 -0.105903803496301,
 -0.5358980946771431,
 -0.31366404280994564,
 0.7777700593006833,
 0.9146442256184394,
 0.48459099849470305,
 0.11080525077159722,
 -0.9656357654018414,
 0.9737279913519716,
 0.5319589257495517,
 0.9956230914517219,
 0.48682123564309765,
 -0.484897902256356,
 0.5791514931370622,
 0.15744970934574964,
 -0.9772162984957723,
 -0.2770327402779192,
 -0.7746338951984411,
 -0.655895619972691,
 -0.1021743587186752,
 0.25698429832699954,
 0.8617085926612364,
 -0.9899871580091143,
 -0.9970349352676108,
 -0.

RDD 还提供了许多便捷的函数，如 `count()` 用来计算数据/容器的大小，`take()` 返回前 `n` 个元素等等。完整的函数列表可以参考[官方文档](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html)。

- *本质来讲.count()代表的是rdd的容器长度，即能够next迭代的次数*

In [19]:
dat.count() 

100

*`take()` `first()` *可以读取一部分内容*

In [21]:
dat.take(5)
dat.first()

0.8414709848078965

23/03/23 17:13:57 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:593)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:592)
	at org.apache.spar

### 3. RDD 文件操作

利用 Numpy 创建一个矩阵，并写入文件：
- *先定义运算，最后最后再进行读取*

In [6]:
import os
import numpy as np
np.set_printoptions(linewidth=100)

np.random.seed(123)
n = 100
p = 5
mat = np.random.normal(size=(n, p))

if not os.path.exists("data"):
    os.makedirs("data", exist_ok=True)
np.savetxt("data/mat_np.txt", mat, fmt="%f", delimiter="\t")

PySpark 读取文件并进行一些简单操作：

In [7]:
file = sc.textFile("data/mat_np.txt")

# 打印矩阵行数
print(file.count())

# 空行
print()

# 打印前5行
text = file.take(5)
print(*text, sep="\n")

100

-1.085631	0.997345	0.282978	-1.506295	-0.578600
1.651437	-2.426679	-0.428913	1.265936	-0.866740
-0.678886	-0.094709	1.491390	-0.638902	-0.443982
-0.434351	2.205930	2.186786	1.004054	0.386186
0.737369	1.490732	-0.935834	1.175829	-1.253881


In [8]:
file.first()

'-1.085631\t0.997345\t0.282978\t-1.506295\t-0.578600'

`file` 的类型也是 RDD。`file` 代表了一些字符串的集合，每个元素是矩阵文件中的一行：

In [9]:
print(type(file))
print(type(file.first()))

<class 'pyspark.rdd.RDD'>
<class 'str'>


我们可以对 RDD 进行变换，使一种元素类型的 RDD 变成另一种元素类型的 RDD。例如，将 `file` 中的每一个字符串变成一个 Numpy 向量，那么变换的结果就是以 Numpy.array 为类型的 RDD。

为此，我们需要先编写一个转换函数：

In [24]:
# str => np.array
def str_to_vec(line):
    # 分割字符串
    str_vec = line.split("\t")
    # 将每一个元素从字符串变成数值型
    num_vec = map(lambda s: float(s), str_vec)
    # 创建 Numpy 向量
    return np.fromiter(num_vec, dtype=float)

print(file.first())
print(str_to_vec(file.first()))

-1.085631	0.997345	0.282978	-1.506295	-0.578600
[-1.085631  0.997345  0.282978 -1.506295 -0.5786  ]


也可以让 Numpy 直接对字符串进行转换：

In [11]:
# str => np.array
def str_to_vec(line):
    # 分割字符串
    str_vec = line.split("\t")
    # 让 Numpy 进行类型转换
    return np.array(str_vec, dtype=float)

print(file.first())
print(str_to_vec(file.first()))

-1.085631	0.997345	0.282978	-1.506295	-0.578600
[-1.085631  0.997345  0.282978 -1.506295 -0.5786  ]


生成新的 RDD：

In [12]:
dat = file.map(str_to_vec)
print(type(dat))
print(type(dat.first()))

<class 'pyspark.rdd.PipelinedRDD'>
<class 'numpy.ndarray'>


RDD 的一般操作都支持：

In [13]:
print(dat.first())
print()
print(dat.take(3))
print()
print(dat.count())

[-1.085631  0.997345  0.282978 -1.506295 -0.5786  ]

[array([-1.085631,  0.997345,  0.282978, -1.506295, -0.5786  ]), array([ 1.651437, -2.426679, -0.428913,  1.265936, -0.86674 ]), array([-0.678886, -0.094709,  1.49139 , -0.638902, -0.443982])]

100


In [32]:
dat = file.map(str_to_vec)
arr, num = dat.map(lambda x: (x,1)).reduce(lambda x,y:(x[0]+y[0],x[1]+y[1]))
print(arr)
print(num)
arr/num

[-9.70826   0.832708 -3.945197 -1.719718 -4.781526]
100


array([-0.0970826 ,  0.00832708, -0.03945197, -0.01719718, -0.04781526])

**参考R：tidyverse中的pipeline操作！**

In [ ]:
n, xsum = sc.textFile("/mat_np.txt").\
    map(str_to_vec).\
    map(lambda x:(1,x)).\
    reduce(lambda x,y:(x[0]+y[0],x[1]+y[1]))
xsum/n

### 4. RDD 分区

RDD 的一个重要功能在于可以分区（分块），从而支持分布式计算。查看 RDD 的分区数：
- 如果逐条处理内容，则没有充分利用分布计算机的本身的计算（空间存储）性能，且会增加频繁通信的通信成本和其他运行成本
- 时间复杂度和空间复杂度的Tradeoff（空间换时间）
- 通过对于内存的配置，极大地利用了本地计算机的局域存储资源计算资源等
- 这些分块后的内容是不是也可以并行计算？是的

In [34]:
file.getNumPartitions() # 根据数据的大小自动划定分区

2

还可以手动指定分区数，从而支持更高的并行度。注意调用 `repartition()` 函数不改变原有 RDD，要使用分区后的 RDD，需要重新赋值：

In [35]:
file_p10 = file.repartition(10)
print(file.getNumPartitions())
print(file_p10.getNumPartitions())

2
10


我们可以按分区对数据进行转换，例如将每个分区的数据转成 Numpy 矩阵。需要使用的函数是 `mapPartitions()`，其接收一个函数作为参数，该函数将对每个分区的**迭代器**进行变换。某些分区可能会是空集，需要做特殊处理。

In [41]:
# Iter[str] => Iter[matrix]
def part_to_mat(iterator):
    # Iter[str] => Iter[np.array]
    iter_arr = map(str_to_vec, iterator)


    # Iter[np.array] => list(np.array)
    dat = list(iter_arr) #dat是由向量（nparray）作为元素组成的列表（list）

# 有的分区可能是空分区
    # list(np.array) => matrix
    if len(dat) < 1:  # Test zero iterator
        mat = np.array([])
    else:
        mat = np.vstack(dat) 

    # matrix => Iter[matrix]
    yield mat # yield可以认为是return[mat]，返回的不是mat本身，而是包含这个的容器

In [39]:
v1 = np.array([1,2,3])
v2 = np.array([3,4,5])
vs = np.vstack([v1,v2])
type(vs)

numpy.ndarray

变换后的结果依然是一个 RDD，但此时元素类型变成了矩阵。

***- 感觉是，分区一旦运行了那个命令，系统就自动把数据分好块了。如果对这些块做操作，那么就是mapPartition，如果直接map就还是单纯的对每行进行操作***

In [44]:
dat_p10 = file_p10.mapPartitions(part_to_mat)
# 原先是包含100行向量的rdd，转换后是包含10个矩阵的rdd
print(type(dat_p10))
print()
print(dat_p10.first()) #第一个分区是空的，请留意这种边缘情况
print()
print(dat_p10.take(3))
print()
print(dat_p10.count())

<class 'pyspark.rdd.PipelinedRDD'>

[]

[array([], dtype=float64), array([[-1.085631,  0.997345,  0.282978, -1.506295, -0.5786  ],
       [ 1.651437, -2.426679, -0.428913,  1.265936, -0.86674 ],
       [-0.678886, -0.094709,  1.49139 , -0.638902, -0.443982],
       [-0.434351,  2.20593 ,  2.186786,  1.004054,  0.386186],
       [ 0.737369,  1.490732, -0.935834,  1.175829, -1.253881],
       [-0.637752,  0.907105, -1.428681, -0.140069, -0.861755],
       [-0.255619, -2.798589, -1.771533, -0.699877,  0.927462],
       [-0.173636,  0.002846,  0.688223, -0.879536,  0.283627],
       [-0.805367, -1.727669, -0.3909  ,  0.573806,  0.338589],
       [-0.01183 ,  2.392365,  0.412912,  0.978736,  2.238143]]), array([[-1.294085, -1.038788,  1.743712, -0.798063,  0.029683],
       [ 1.069316,  0.890706,  1.754886,  1.495644,  1.069393],
       [-0.772709,  0.794863,  0.314272, -1.326265,  1.417299],
       [ 0.807237,  0.04549 , -0.233092, -1.198301,  0.199524],
       [ 0.468439, -0.831155,  1.16

23/03/30 17:15:29 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:593)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:592)
	at org.apache.spar

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 64715)
Traceback (most recent call last):
  File "/Users/xinby/opt/anaconda3/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/xinby/opt/anaconda3/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/xinby/opt/anaconda3/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/xinby/opt/anaconda3/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/xinby/Library/Spark/python/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/Users/xinby/Library/Spark/python/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/Users/xinby/Library/Spark/python/pyspark/accumulators.py", line 257

我们可以用 `filter()` 过滤掉空的分区：

In [ ]:
dat_p10_nonempty = dat_p10.filter(lambda x: x.shape[0] > 0)

print(type(dat_p10_nonempty))
print()
print(dat_p10_nonempty.first())
print()
print(dat_p10_nonempty.count())

### 5. RDD 操作案例 - 求列和

np.array 版本的 RDD 求矩阵的列和：

In [ ]:
dat.reduce(lambda x1, x2: x1 + x2)

从输入矩阵文件开始，将操作串联：

In [ ]:
file.map(str_to_vec).reduce(lambda x1, x2: x1 + x2)

使用分区版本的 RDD，先在每个分区上求列和：

In [ ]:
sum_part = dat_p10_nonempty.map(lambda x: np.sum(x, axis=0))
sum_part.collect()

再将分区结果汇总：

In [ ]:
sum_part.reduce(lambda x1, x2: x1 + x2)

从输入矩阵文件开始，将操作串联：

In [ ]:
file.repartition(10).\
    mapPartitions(part_to_mat).\
    filter(lambda x: x.shape[0] > 0).\
    map(lambda x: np.sum(x, axis=0)).\
    reduce(lambda x1, x2: x1 + x2)

使用真实值检验：

In [ ]:
np.sum(mat, axis=0)

### 6. RDD 操作案例 - 矩阵乘法

模拟数据和真实值：

In [ ]:
np.random.seed(123)
v = np.random.uniform(size=p)
res = mat.dot(v)
res

np.array 版 RDD：

In [ ]:
res1 = dat.map(lambda x: x.dot(v)).collect()
res1[:10]

分区版 RDD：

In [ ]:
res_part = dat_p10_nonempty.map(lambda x: x.dot(v)).collect()
res_part

拼接分区结果：

In [ ]:
np.concatenate(res_part)

关闭 Spark 连接：

In [ ]:
sc.stop()